# Segmenting and Clustering Neighborhoods in Toronto
### Part 1: Scraping Postal Codes from Wikipidea

In [1]:
import requests # library to handle requests
import lxml.html as lh
import bs4 as bs
import urllib.request

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

from geopy.geocoders import Nominatim
import folium

In [2]:
## Source URL
url   = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
page  = urllib.request.urlopen(url).read()
soup  = bs.BeautifulSoup(page,'lxml')
table = soup.find("table",class_="wikitable")
header = [head.findAll(text=True)[0].strip() for head in table.find_all("th")]
data   = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]for tr in table.find_all("tr")]
data= [row for row in data if len(row) == 3]
df = pd.DataFrame(data,columns=header)    

In [4]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)

In [6]:
df= df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.columns = ['Postcode', 'Borough', 'Neighbourhood']

In [7]:
df['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)

In [8]:
df['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)

In [9]:
df.shape

(103, 3)

## part 2

In [10]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2019-03-06 22:14:09--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2019-03-06 22:14:09--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-03-06 22:14:11--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-03-06 22:14:12--  https://ibm.box.com/public/static/9afzr83pps4pwf2sm

In [11]:
df_cord = pd.read_csv('GeoCord.csv') # Read the csv data
df_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# Create Latitude and Longitude columns in df
df['Latitude'] = np.nan
df['Longitude'] = np.nan

# For each postcode in df, find corresponding coordinates in df_cord and assign it to df
for idx in df.index:
    cord_idx = df_cord['Postal Code'] == df.loc[idx, 'Postcode']
    df.at[idx, 'Latitude'] = df_cord.loc[cord_idx, 'Latitude'].values
    df.at[idx, 'Longitude'] = df_cord.loc[cord_idx, 'Longitude'].values
df.to_csv("df2.csv")
# Display the results
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [13]:
from geopy.geocoders import Nominatim
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


In [14]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto